In [1]:
%matplotlib ipympl

In [2]:
data = ['val-id', 'test-id', 'test-ood']
layers = ['layer1', 'layer2', 'layer3', 'layer4']

In [3]:
import os
import numpy as np

for layer in layers:
    print(np.load(os.path.join('out', f'{data[2]}_{layer}.npy')).shape)

(2, 8000, 256)
(2, 8000, 512)
(2, 8000, 1024)
(2, 8000, 2048)


### Plotting In-distribution Val conductance for different class members

In [4]:
import matplotlib.pyplot as plt
import os
import numpy as np

data_name = 'val-id'
layer_name = 'layer4'

attribution = np.load(os.path.join('out', f'{data_name}_{layer_name}.npy'))
labels = np.load(os.path.join('out', f'{data_name}_{layer_name}_labels.npy'))

def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)


fig, ax = plt.subplots(figsize=(40, 24))
for ix, (label, offset) in enumerate(zip(np.unique(labels), [-0.4, 0.4])):
    color = 'blue' if ix else 'red'
    label_mask = labels == label
    bps = ax.boxplot(attribution[label_mask].tolist(), positions=np.arange(label_mask.sum()) * 2.0 + offset, sym='', widths=0.6)
    set_box_color(bps, color)
    
    plt.plot([], c=color, label=label)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plotting In-distribution TEST conductance for different class members

In [5]:
import matplotlib.pyplot as plt
import os
import numpy as np

data_name = 'test-id'
layer_name = 'layer4'

attribution = np.load(os.path.join('out', f'{data_name}_{layer_name}.npy'))
labels = np.load(os.path.join('out', f'{data_name}_{layer_name}_labels.npy'))

def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)


fig, ax = plt.subplots(figsize=(40, 24))
for ix, (label, offset) in enumerate(zip(np.unique(labels), [-0.4, 0.4])):
    color = 'blue' if ix else 'red'
    label_mask = labels == label
    bps = ax.boxplot(attribution[label, label_mask].tolist(), positions=np.arange(label_mask.sum()) * 2.0 + offset, sym='', widths=0.6)
    set_box_color(bps, color)
    
    plt.plot([], c=color, label=label)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Note that usually we woulnd't know the red and blue separation for the above since this is test

### Now let's look at the class activations for class 0 and 1 without knowing the true labels on our out of distribtion images

In [6]:
import matplotlib.pyplot as plt
import os
import numpy as np

data_name = 'test-ood'
layer_name = 'layer1'

attribution = np.load(os.path.join('out', f'{data_name}_{layer_name}.npy'))
# labels = np.load(os.path.join('out', f'{data_name}_{layer_name}_labels.npy'))

def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)

    
sample = np.random.choice(np.arange(attribution.shape[1]), size=1000, replace=False)
fig, ax = plt.subplots(figsize=(40, 24))
for label, offset in enumerate([-0.4, 0.4]):
    color = 'blue' if label else 'red'
    bps = ax.boxplot(attribution[label, sample].tolist(), positions=np.arange(len(sample)) * 2.0 + offset, sym='', widths=0.6)
    set_box_color(bps, color)
    
    plt.plot([], c=color, label=label)
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(1000, 256)
(1000, 256)


In [84]:
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
import numpy as np


outliers_fraction = 0.
classifier = IsolationForest(behaviour='new', contamination=outliers_fraction, random_state=42)

X = [[0], [0.44], [0.45], [0.46], [1]]
X = np.array(X)

classifier.fit(X)

# clf = OneClassSVM(gamma='auto', nu=0.001).fit(X)
classifier.predict(X)

array([1, 1, 1, 1, 1])

In [107]:
from sklearn.svm import OneClassSVM

data_name = 'val-id'
layer_name = 'layer4'

attribution = np.load(os.path.join('out', f'{data_name}_{layer_name}.npy'))
labels = np.load(os.path.join('out', f'{data_name}_{layer_name}_labels.npy'))

dog_classifier = IsolationForest(behaviour='new', contamination=0., random_state=42)
dog_classifier = classifier.fit(attribution[labels == 0])

cat_classifier = IsolationForest(behaviour='new', contamination=0., random_state=42)
cat_classifier = classifier.fit(attribution[labels == 1])

In [108]:
pred_on_train = classifier.predict(attribution)
np.unique(pred_on_train, return_counts=True)

(array([-1,  1]), array([ 63, 937]))

In [109]:
data_name = 'test-id'
layer_name = 'layer4'

test_id_attribution = np.load(os.path.join('out', f'{data_name}_{layer_name}.npy'))
test_id_labels = np.load(os.path.join('out', f'{data_name}_{layer_name}_labels.npy'))
print(test_id_attribution.shape)
print(test_id_attribution[0].shape)
print(test_id_labels.shape)

# prediction for in distribution of each attribution, where attributions are measured against class 0, regardless of the true class.
# for all dog attributions, is the attribution in or out of distribution. ie is the image a dog
pred_in_zero = dog_classifier.predict(test_id_attribution[0])
# opposite
pred_in_one = cat_classifier.predict(test_id_attribution[1])

(2, 2000, 2048)
(2000, 2048)
(2000,)


In [110]:
np.unique(pred_in_zero, return_counts=True)

(array([-1,  1]), array([ 173, 1827]))

In [111]:
from sklearn.metrics import classification_report

# predicted out of distribution are given a FALSE. Else given a TRUE.
pred_in_zero[pred_in_zero == -1] = 0

# boolean list: true if dog, false otherwise
dog_images_mask = (test_id_labels == 0)
print(classification_report(dog_images_mask, pred_in_zero))

              precision    recall  f1-score   support

       False       0.86      0.15      0.25      1000
        True       0.53      0.97      0.69      1000

    accuracy                           0.56      2000
   macro avg       0.69      0.56      0.47      2000
weighted avg       0.69      0.56      0.47      2000



In [112]:
from sklearn.metrics import classification_report

pred_in_one[pred_in_one == -1] = 0

cat_images_mask = (test_id_labels == 1)
print(classification_report(cat_images_mask, pred_in_one))

              precision    recall  f1-score   support

       False       0.10      0.01      0.02      1000
        True       0.48      0.90      0.62      1000

    accuracy                           0.45      2000
   macro avg       0.29      0.45      0.32      2000
weighted avg       0.29      0.45      0.32      2000

